In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(1)

In [4]:
Meta.df_comb.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_76d55d9d0,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_96d282ea9,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_7d8a7c29d,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_4d4401491,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_8f5ded0b7,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


## Training

In [6]:
name = 'baseline_full_stg1_xresnet18_imagenet'

In [7]:
dbch = get_data(512, 128, splits=Meta.splits_stg1, mean=imagenet_stats[0], std=imagenet_stats[1])
learn = get_learner(dbch, xse_resnext18, pretrained=True)

## Training

In [8]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

KeyboardInterrupt: 

In [ ]:
learn.dls = get_data(512, 256, splits=Meta.splits_stg1, mean=imagenet_stats[0], std=imagenet_stats[1])
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

In [ ]:
learn.dls = get_data(256, 384, splits=Meta.splits_stg1, img_dir=path_jpg, mean=imagenet_stats[0], std=imagenet_stats[1])
do_fit(learn, 4, 4e-4)
learn.save(f'runs/{name}-3')

## Submission

In [11]:
learn.load(f'runs/{name}-3')

In [12]:
# tst_fns = df_tst.index.values

In [13]:
sub_fn = f'subm/{name}-3'

In [14]:
learn.dls = get_test_data(Meta.df_tst, bs=512, sz=384)

In [15]:
# tst_bs = 256
# tst_sz = 384
# tst_splits = [L.range(tst_fns), L.range(tst_fns)]
# tst_dbch = get_data_gen(tst_fns, bs=tst_bs, img_tfm=get_pil_fn(path/'tst_jpg'), sz=tst_sz, splits=tst_splits, test=True)
# learn.dls = tst_dbch

In [16]:
preds,targs = learn.get_preds()

In [17]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [18]:
FileLink(f'{sub_fn}.csv')

/home/ashaw/kaggle/rsna_retro/subm/baseline_full_stg1-3.csv

In [19]:
api.competition_submit(f'{sub_fn}.csv', 'Training on Stage 1 (384), Valid on Stage 2. - 0.084161', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.72MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [25]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14258280,
 'totalBytes': 27277209,
 'date': '2020-01-22T06:54:06.2Z',
 'description': 'Training on Stage 1 (384), Valid on Stage 2. - 0.084161',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1-3.csv',
 'publicScore': '0.61200',
 'privateScore': '0.06511',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14258280/14258280.raw'}